<a href="https://colab.research.google.com/github/ttogle918/NLU_3-/blob/main/%EC%9D%B4%EC%88%98%EC%B2%A0_sts_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP

## KLUE-STS - Semantic Textual Similarity

- 입력으로 주어진 두 문장간의 의미 동등성을 수치로 표현하는 것이 목표.
- 두 입력 문장 간 유사도를 0 ~ 5 점으로 유사도 3.0을 기준으로 유사하다, 유사하지 않다로 라벨링 후 평가함.
- 평가방법 : F1 score

sts
An example of 'train' looks as follows.

```
{'guid': 'klue-sts-v1_train_00000',
 'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1},
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'source': 'airbnb-rtt'}

```



In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.3 MB/s 
     |████████████████████████████████| 6.6 MB 13.7 MB/s 
     |████████████████████████████████| 596 kB 44.2 MB/s 
     |████████████████████████████████| 86 kB 2.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 12.7 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 212 kB 62.0 MB/s 
     |████████████████████████████████| 1.1 MB 43.8 MB/s 
     |████████████████████████████████| 140 kB 54.6 MB/s 
     |████████████████████████████████| 127 kB 81.6 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████

In [1]:
import os
import sys
import pandas as pd
import numpy as np 
import torch
import random

from datetime import datetime
from datasets import load_dataset

In [2]:
# seed
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

cpu


In [3]:
# 사전 학습 언어모델
model_name = "klue/roberta-base"

##데이터셋 불러오기

In [4]:
dataset = load_dataset('klue', 'sts')

Reusing dataset klue (/root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 11668
    })
    validation: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 519
    })
})

In [6]:
# DataFrame으로 변환


# Train
sentence1, sentence2, labels = [], [], []

for data in dataset['train'] :
  sentence1.append(data['sentence1'])
  sentence2.append(data['sentence2'])
  labels.append(data['labels']['real-label'])
  # labels.append(data['labels'])
df = pd.DataFrame({'sentence1' : sentence1, 'sentence2' : sentence2, 'labels' : labels})

In [7]:
df

,sentence1,sentence2,labels
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,3.714286
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0.000000
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0.333333
3,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0.571429
4,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,4.714286
...,...,...,...
11663,일단 방은 사진이랑 똑같고 매우 깨끗합니다.,"우선, 방은 그림과 같고 매우 깨끗합니다.",4.000000
11664,생일이 삼월이라는 친구 이름이 뭐야?,"운동하러 갈 시간은 밥 먹기 전이 좋아, 먹은 후가 좋아?",0.000000
11665,그녀는 지속적으로 나에게 그녀가 만든 새로운 규칙을 매일 매일 이메일로 보냈다.,그녀는 자신이 만든 새로운 규칙을 매일 끊임없이 제게 이메일로 보냈습니다.,3.666667
11666,개 1마리 고양이 3마리 너무 귀여워요!,개 한 마리와 고양이 세 마리는 정말 귀여워요!,4.714286


In [8]:
# Validation

sentence3, sentence4, target_labels = [], [], []

for data in dataset['validation'] :
  sentence3.append(data['sentence1'])
  sentence4.append(data['sentence2'])
  target_labels.append(data['labels']['real-label'])
  # labels.append(data['labels'])
val_df = pd.DataFrame({'sentence3' : sentence3, 'sentence4' : sentence4, 'target_labels' : target_labels})

In [9]:
val_df

,sentence3,sentence4,target_labels
0,무엇보다도 호스트분들이 너무 친절하셨습니다.,"무엇보다도, 호스트들은 매우 친절했습니다.",4.857143
1,주요 관광지 모두 걸어서 이동가능합니다.,위치는 피렌체 중심가까지 걸어서 이동 가능합니다.,1.428571
2,학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018...,영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년...,1.285714
3,"다만, 도로와 인접해서 거리의 소음이 들려요.","하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다.",3.714286
4,형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요.,가족 모임 일정은 바꾸지 말도록 하십시오.,2.500000
...,...,...,...
514,"문체부는 이를 연차적으로 확대, 시행해 학교운동부와 스포츠클럽 간의 연계를 강화한다.",문화체육관광부는 학교스포츠학과와 스포츠클럽의 연계성을 강화하기 위해 매년 이 프로그...,2.200000
515,일단 정확한 정보와 빠른 답변이 정말 좋았습니다.,호스트의 빠른 답변과 유용한 정보들이 정말 좋습니다.,2.833333
516,게스트에 대한 배려가 묻어나는 시설들이었습니다.,우선 공간에 대한 센스가 돋보이는 곳이었습니다.,0.333333
517,밤하늘을 배경으로 ‘비대면 드론쇼’도 펼쳐진다.,‘비대면 실감형 문화공연 플랫폼’ 개념도.,0.333333


추가 데이터셋 kor_nlu_data

In [ ]:
# kakaobrain/KorNLUDatasets 
#kor_nlu_data = load_dataset("kor_nlu", "sts")

###전처리

In [10]:
df.duplicated(['sentence1', 'sentence2']).sum()

7

In [11]:
df = df.drop_duplicates(['sentence1','sentence2'], keep='first', ignore_index=True)

In [12]:
df

,sentence1,sentence2,labels
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,3.714286
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0.000000
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0.333333
3,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.",0.571429
4,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,4.714286
...,...,...,...
11656,일단 방은 사진이랑 똑같고 매우 깨끗합니다.,"우선, 방은 그림과 같고 매우 깨끗합니다.",4.000000
11657,생일이 삼월이라는 친구 이름이 뭐야?,"운동하러 갈 시간은 밥 먹기 전이 좋아, 먹은 후가 좋아?",0.000000
11658,그녀는 지속적으로 나에게 그녀가 만든 새로운 규칙을 매일 매일 이메일로 보냈다.,그녀는 자신이 만든 새로운 규칙을 매일 끊임없이 제게 이메일로 보냈습니다.,3.666667
11659,개 1마리 고양이 3마리 너무 귀여워요!,개 한 마리와 고양이 세 마리는 정말 귀여워요!,4.714286


### 전처리 후 Dataset

### 토큰화

In [13]:
import torch
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
model = AutoModel.from_pretrained(model_name)

In [15]:
# Train Dataset을 학습과 검증 셋으로 분리

from torch.utils.data import RandomSampler, SequentialSampler, random_split
train_sample = dataset.shape[0]

n_train = int(train_sample*0.9)
n_valid = train_sample - n_train #int(n_train_sample*0.1)
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

print(f"Train dataset len: {len(train_dataset)}")
print(f"Valid dataset len: {len(valid_dataset)}")

KeyError: ignored

In [60]:
train_dataset[0]

{'labels': 4.571428571428571,
 'sentence1': '숙소위치는대중교통이용하기편리한위치였습니다',
 'sentence2': '그숙소는대중교통을이용하기에편리한장소였습니다'}

DataLoader

In [46]:
from torch.utils.data.dataloader import DataLoader
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size = 16
)

In [59]:
train_dataloader